In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

def iniciar_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    return webdriver.Chrome(options=options)

def info_no_texto(texto_antes, texto_completo):
    info = None
    if texto_completo:
        match = re.search(texto_antes, texto_completo)
        if match:
            info = match.group(1)
    return info

def cidade_e_cep(driver):
    texto_cep_cidade = None
    try:
        try:
            link_universidade = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div[1]/div/h5/a'
            )
            url_universidade = link_universidade.get_attribute('href')
        except:
            url_universidade = None

        if url_universidade:
            driver.get(url_universidade)
            
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#app"))
            )
            
            time.sleep(2)

            try:
                texto_cep_cidade = driver.find_element(
                    By.XPATH,
                    '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[2]/div/div/div[2]/span'
                ).text
            except:
                texto_cep_cidade = None

            driver.back()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#app"))
            )
            time.sleep(2)
    except Exception as e:
        print(f"Sem infos de cidade e cep do curso")
        
    return texto_cep_cidade
    
def extrair_dados_curso(driver, id_curso):
    info_curso = {}
    url = f"https://sucupira.capes.gov.br/programas/detalhamento/{id_curso}"
    
    try:
        driver.get(url)
        
        # Espera até que o conteúdo dentro de #app seja carregado
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app"))
        )
        
        time.sleep(2)
        
        try:
            codigo_curso = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div[1]/div/div/div[1]/div/div[2]/div/div/span'
            ).text
        except:
            codigo_curso = None

        info_curso['id'] = id_curso
        info_curso['codigo_curso'] = codigo_curso

        try:
            area_basica = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div[2]/div/div/div[3]/div/div[2]/div/div/span'
            ).text
        except:
            area_basica = None

        info_curso['area_basica'] = area_basica

        try:
            area_avaliacao = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div[2]/div/div/div[2]/div/div[2]/div/div/span'
            ).text
        except:
            area_avaliacao = None

        info_curso['area_avaliacao'] = area_avaliacao
  
        try:
            situacao = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div[3]/div/div/div[3]/div/div[2]/div/div/span'
            ).text
        except:
            situacao = None

        info_curso['situacao'] = situacao

        try:
            texto_inicial = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[3]/div/span'
            ).text
        except:
            texto_inicial = None

        inicio = info_no_texto(r'INÍCIO EM (\d{4})', texto_inicial)
        info_curso['inicio'] = inicio

        nota = info_no_texto(r'NOTA (\d+)', texto_inicial)
        info_curso['nota'] = nota

        try:
            universidade = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[2]/div/div/div[1]/span'
            ).text
        except:
            universidade = None

        info_curso['universidade'] = universidade

        texto_cep_cidade = cidade_e_cep(driver)
        cep = info_no_texto(r'CEP (\d{8})', texto_cep_cidade)

        cidade = None
        if texto_cep_cidade:
            partes = [parte.strip() for parte in texto_cep_cidade.split('-')]
        
            if len(partes) >= 2:
                cidade = partes[1]
                
        info_curso['cep'] = cep
        info_curso['cidade'] = cidade

        doutorado = False
        mestrado = False
        info_doutorado = None
        info_mestrado = None

        container_1 = {
            'situacao': None,
            'nota': None,
            'codigo': None
        }  

        container_2 = {
            'situacao': None,
            'nota': None,
            'codigo': None
        }  

        try:
            wait = WebDriverWait(driver, 10)
            container_mestrado_doutorado = wait.until(EC.element_to_be_clickable((
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]'
            )))
            container_mestrado_doutorado.click()
            
            time.sleep(2)              
                
            try:
                situacao_container = driver.find_element(
                    By.XPATH,
                    '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[1]/div/div[2]/div/div/div[5]/span'
                ).text
            except:
                situacao_container = None
            
            try:
                texto_nota = driver.find_element(
                    By.XPATH,
                    '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[1]/div/div[2]/div/div/div[1]/div'
                ).text
            except:
                texto_nota = None
            
            nota_container = info_no_texto(r'Nota (\d+)', texto_nota)
            
            try:
                codigo_container = driver.find_element(
                    By.XPATH,
                    '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[1]/div/div[2]/div/div/div[2]/span/strong'
                ).text
            except:
                codigo_container = None
            
            container_1['situacao'] =  situacao_container
            container_1['nota'] = nota_container
            container_1['codigo'] = codigo_container

            try:
                situacao_container = driver.find_element(
                    By.XPATH,
                    '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[2]/div/div[2]/div/div/div[5]/span'
                ).text
            except:
                situacao_container = None
        
            try:
                texto_nota = driver.find_element(
                    By.XPATH,
                    '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[2]/div/div[2]/div/div/div[1]/div'
                ).text
            except:
                texto_nota = None
        
            nota_container = info_no_texto(r'Nota (\d+)', texto_nota)
        
            try:
                codigo_container = driver.find_element(
                    By.XPATH,
                    '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[2]/div/div[2]/div/div/div[2]/span/strong'
                ).text
            except:
                codigo_container = None
        
            container_2['situacao'] = situacao_container
            container_2['nota'] = nota_container
            container_2['codigo'] = codigo_container
        except Exception as e:
            print(f"Sem infos mestrado e doutorado")
        
        try:
            titulo_mestrado_ou_doutorado = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[1]/div/div[1]/div/h6'
            ).text
        except:
            titulo_mestrado_ou_doutorado = None

        
        if titulo_mestrado_ou_doutorado: 
            if "MESTRADO" in titulo_mestrado_ou_doutorado:
                info_mestrado = container_1
                mestrado = True
            else:
                if "DOUTORADO" in titulo_mestrado_ou_doutorado:
                    info_doutorado = container_1
                    doutorado = True
        
        try:
            titulo_mestrado_ou_doutorado = driver.find_element(
                By.XPATH,
                '//*[@id="main-content"]/div/div/div[2]/div[2]/div/div[2]/div[4]/div/div[2]/div/div/div/div[5]/div/div[2]/div/div[2]/div/div[1]/div/h6'
            ).text
        except:
            titulo_mestrado_ou_doutorado = None
        
        if titulo_mestrado_ou_doutorado: 
            if "MESTRADO" in titulo_mestrado_ou_doutorado:
                info_mestrado = container_2
                mestrado = True
            else:
                if "DOUTORADO" in titulo_mestrado_ou_doutorado:
                    info_doutorado = container_2
                    doutorado = True

        info_curso['doutorado'] = doutorado
        info_curso['mestrado'] = mestrado

        info_curso['info_doutorado'] = info_doutorado
        info_curso['info_mestrado'] = info_mestrado

        return info_curso
        
    except Exception as e:
        print(f"Erro ao acessar o curso {id_curso}: {str(e)}")
        return None

def main():
    driver = iniciar_driver()
    info_cursos = []
    
    try:
        for i in range(1000, 2000):
            info_curso = extrair_dados_curso(driver, i)
            if info_curso and info_curso['codigo_curso'] is not None:
                info_cursos.append(info_curso)
            if len(info_cursos) >= 500:
                break;
        
        if info_cursos:
            df_final = pd.DataFrame(info_cursos)
            df_final.to_csv('cursos_sucupira.csv', index=False)
            print("\nDados salvos em cursos_sucupira.csv")
            
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem infos mestrado e doutorado
Sem info